In [1]:
# Using pandas, sqlalchemy and numpy in this analysis
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [2]:
# Importing csv files to create dataframe
us_counties_file = "live/us-counties.csv"
counties_df = pd.read_csv(us_counties_file)
counties_df.head()

,date,county,state,fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths
0,2020-08-01,Autauga,Alabama,1001.0,1015,21.0,972.0,20.0,43.0,1.0
1,2020-08-01,Baldwin,Alabama,1003.0,3101,22.0,3056.0,21.0,45.0,1.0
2,2020-08-01,Barbour,Alabama,1005.0,598,5.0,550.0,5.0,48.0,0.0
3,2020-08-01,Bibb,Alabama,1007.0,363,2.0,355.0,2.0,8.0,0.0
4,2020-08-01,Blount,Alabama,1009.0,767,3.0,685.0,3.0,82.0,0.0


### Transform County DataFrame ###

In [4]:
# Reset all float numbers to integer
counties_df['fips'] = counties_df['fips'].astype('O')
counties_df.loc[counties_df['fips'].notnull(), 'fips'] = counties_df.loc[counties_df['fips'].notnull(), 'fips'].astype(int)


counties_df["confirmed_cases"] = counties_df["confirmed_cases"].astype('O')
counties_df.loc[counties_df["confirmed_cases"].notnull(), "confirmed_cases"] = counties_df.loc[counties_df["confirmed_cases"].notnull(), "confirmed_cases"].astype(int)

counties_df["confirmed_deaths"] = counties_df["confirmed_deaths"].astype('O')
counties_df.loc[counties_df["confirmed_deaths"].notnull(), "confirmed_deaths"] = counties_df.loc[counties_df["confirmed_deaths"].notnull(), "confirmed_deaths"].astype(int)


# Create a filtered dataframe from specific columns
# counties_df.info()

counties_cols = ["fips","county", "state", "confirmed_cases","confirmed_deaths"]
counties_transformed= counties_df[counties_cols].copy()

# Rename the column headers
counties_transformed = counties_transformed.rename(columns={"fips": "countyfp",
                                                              "county": "county",
                                                              "state": "state",
                                                                "confirmed_cases":"confirmed_cases",
                                                                "confirmed_deaths":"confirmed_deaths"})

# Drop NA rows
counties_transformed.dropna(inplace=True)

# Reset the index column
counties_transformed.set_index("countyfp", inplace=True)
counties_transformed.index = counties_transformed.index.astype('int64')

counties_transformed

,county,state,confirmed_cases,confirmed_deaths
countyfp,,,,
1001,Autauga,Alabama,972,20
1003,Baldwin,Alabama,3056,21
1005,Barbour,Alabama,550,5
1007,Bibb,Alabama,355,2
1009,Blount,Alabama,685,3
...,...,...,...,...
55133,Waukesha,Wisconsin,3488,53
55135,Waupaca,Wisconsin,350,14
55137,Waushara,Wisconsin,99,0


### Transform Mask-Usage Dataframe###

In [5]:
# Importing csv file to create dataframe

mask_use_file = "mask-use/mask-use-by-county.csv"
mask_use_df = pd.read_csv(mask_use_file)
mask_use_df.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


In [6]:
# Rename the columns from caps to lower cases
mask_use_df = mask_use_df.rename(columns={"COUNTYFP": "countyfp",
                                          "NEVER": "never",
                                          "RARELY": "rarely",
                                          "SOMETIMES":"sometimes",
                                          "FREQUENTLY":"frequently",
                                          "ALWAYS":"always"})


# Convert all float numbers to percentages

mask_use_df['never']= mask_use_df['never'].astype(float).map("{:,.2f}%".format)

mask_use_df['rarely']= mask_use_df['rarely'].astype(float).map("{:,.2f}%".format)

mask_use_df['sometimes']= mask_use_df['sometimes'].astype(float).map("{:,.2f}%".format)

mask_use_df['frequently']= mask_use_df['frequently'].astype(float).map("{:,.2f}%".format)

mask_use_df['always']= mask_use_df['always'].astype(float).map("{:,.2f}%".format)

#mask_use_df.set_index("countyfp", inplace=True)

mask_use_df

,countyfp,never,rarely,sometimes,frequently,always
0,1001,0.05%,0.07%,0.13%,0.29%,0.44%
1,1003,0.08%,0.06%,0.10%,0.32%,0.44%
2,1005,0.07%,0.12%,0.12%,0.20%,0.49%
3,1007,0.02%,0.03%,0.10%,0.28%,0.57%
4,1009,0.05%,0.11%,0.18%,0.19%,0.46%
...,...,...,...,...,...,...
3137,56037,0.06%,0.29%,0.23%,0.15%,0.27%
3138,56039,0.10%,0.16%,0.16%,0.25%,0.34%
3139,56041,0.10%,0.28%,0.15%,0.21%,0.26%
3140,56043,0.20%,0.15%,0.07%,0.28%,0.29%


In [7]:
# reset index column
mask_use_df.set_index("countyfp", inplace=True)

In [8]:
# soft to see the dataframe
mask_use_df.sort_values(['countyfp'])

,never,rarely,sometimes,frequently,always
countyfp,,,,,
1001,0.05%,0.07%,0.13%,0.29%,0.44%
1003,0.08%,0.06%,0.10%,0.32%,0.44%
1005,0.07%,0.12%,0.12%,0.20%,0.49%
1007,0.02%,0.03%,0.10%,0.28%,0.57%
1009,0.05%,0.11%,0.18%,0.19%,0.46%
...,...,...,...,...,...
56037,0.06%,0.29%,0.23%,0.15%,0.27%
56039,0.10%,0.16%,0.16%,0.25%,0.34%
56041,0.10%,0.28%,0.15%,0.21%,0.26%


<!-- Create database connection -->

### Create Database Connection ###

In [9]:
# Connect to postgres by create engine 
connection_string = "postgres:postgres@localhost:5432/covidmask_db"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
# Confirm table names
engine.table_names()

['county', 'mask']

### Load DataFrames into Database ###

In [11]:
#county data load into database
counties_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)

In [12]:
# mask data load into database
mask_use_df.to_sql(name='mask', con=engine, if_exists='append', index=True)